# Case Intro
Term deposits are a major source of income for a bank. A term deposit is a cash investment held at a financial institution. Your money is invested for an agreed rate of interest over a fixed amount of time, or term. The bank has various outreach plans to sell term deposits to their customers such as email marketing, advertisements, telephonic marketing, and digital marketing.

Telephonic marketing campaigns still remain one of the most effective way to reach out to people. However, they require huge investment as large call centers are hired to actually execute these campaigns. Hence, it is crucial to identify the customers most likely to convert beforehand so that they can be specifically targeted via call.

The data is related to direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe to a term deposit (variable y).

Content
The data is related to the direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed by the customer or not. The data folder contains two datasets:-

Bank.csv: 45,211 rows and 18 columns ordered by date (from May 2008 to November 2010)

Detailed Column Descriptions
bank client data:

1 - age (numeric)

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
"blue-collar","self-employed","retired","technician","services")

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

6 - balance: average yearly balance, in euros (numeric)

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")
# related with the last contact of the current campaign:
9 - contact: contact communication type (categorical: "unknown","telephone","cellular")
10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", …, "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

# other attributes:
13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)

15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

Output variable (desired target):

17 - y - has the client subscribed a term deposit? (binary: "yes","no")

Missing Attribute Values: None


In [11]:
%pip install optuna

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Obtaining dependency information for optuna from https://files.pythonhosted.org/packages/15/da/68883911855d8b4d521f9a370e4e6aab8232b91c1d8d5a8348c4680c6642/optuna-3.6.1-py3-none-any.whl.metadata
  Obtaining dependency information for alembic>=1.5.0 from https://files.pythonhosted.org/packages/7f/50/9fb3a5c80df6eb6516693270621676980acd6d5a9a7efdbfa273f8d616c7/alembic-1.13.1-py3-none-any.whl.metadata
  Obtaining dependency information for colorlog from https://files.pythonhosted.org/packages/f3/18/3e867ab37a24fdf073c1617b9c7830e06ec270b1ea4694a624038fc40a03/colorlog-6.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for Mako from https://files.pythonhosted.org/packages/2b/8d/9f11d0b9ac521febb806e7f30dc5982d0f4f5821217712c59005fbc5c1e3/Mako-1.3.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/380.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/380.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/380.1 kB ?

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('https://raw.githubusercontent.com/ogut77/DataScience/main/data/Bank.csv',sep = ';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [37]:
print(df.shape)
df.info()
df.isnull().sum()

(45211, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [39]:
#For object check the data
for cn in df.columns:
  if(df[cn].dtype==object):
    print(df[cn].value_counts())


job
blue-collar      9732
management       9458
technician       7597
admin.           5171
services         4154
retired          2264
self-employed    1579
entrepreneur     1487
unemployed       1303
housemaid        1240
student           938
unknown           288
Name: count, dtype: int64
marital
married     27214
single      12790
divorced     5207
Name: count, dtype: int64
education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64
default
no     44396
yes      815
Name: count, dtype: int64
housing
yes    25130
no     20081
Name: count, dtype: int64
loan
no     37967
yes     7244
Name: count, dtype: int64
contact
cellular     29285
unknown      13020
telephone     2906
Name: count, dtype: int64
month
may    13766
jul     6895
aug     6247
jun     5341
nov     3970
apr     2932
feb     2649
jan     1403
oct      738
sep      579
mar      477
dec      214
Name: count, dtype: int64
poutcome
unknown    36959
failure     4901
other  

In [40]:
def Encoder(df):
          from sklearn import preprocessing
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = preprocessing.LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df


In [41]:
df=Encoder(df)
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3,1
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3,1
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2,1
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3,0


In [42]:
y = df['y'] #Output
X = df.drop('y',axis=1)
X

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3


In [45]:
from sklearn.model_selection import  train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)




Q1)Using  Random Forest,XGBoost, Light GBM and Gradient Boosting Classifier with default parameters (no parameter specifications except random_state) calculate Accuracy on Test data. Which method gives the best accuracy on test data

In [59]:
rfc = RandomForestClassifier(random_state=17)
xgb_model = xgb.XGBClassifier(random_state=17)
lgbm_model = LGBMClassifier(random_state=17)
gbc = GradientBoostingClassifier(random_state=17)

rfc.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
gbc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
xgb_pred = xgb_model.predict(X_test)
lgbm_pred = lgbm_model.predict(X_test)
gbc_pred = gbc.predict(X_test)

rfc_acc = accuracy_score(y_test, rfc_pred)
xgb_acc = accuracy_score(y_test, xgb_pred)
lgbm_acc = accuracy_score(y_test, lgbm_pred)
gbc_acc = accuracy_score(y_test, gbc_pred)

models = ['Random Forest', 'XGBoost', 'LightGBM', 'Gradient Boosting']
accuracies = [rfc_acc, xgb_acc, lgbm_acc, gbc_acc]

best_model_idx = accuracies.index(max(accuracies))
best_model = models[best_model_idx]

print('Model with best accuracy on test data:', best_model)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 3777 3778 3779], got [  2383   2442   2444 ...  93813  96493 101561]

Q2) Using optuna hyperparmeter optimization technique and 100 trial

 a)find best methods with  parameters  using Cross validation (CV=3) technique for the range of   parameters below. What are the best parameters for the method with highest cross validation accuracy?

"max_depth": range(2, 16), "max_features": range(2, 16)

 b)Evaluate the performance of the  method with highest cross validation accuracy on test data.What is the accuracy value?


In [47]:
#a)
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

def objective(X, y):
    def _objective(trial):

        skf = StratifiedKFold(n_splits=3)

        max_depth = trial.suggest_int('max_depth', 2, 15)
        max_features = trial.suggest_int('max_features', 2, 15)

        model = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)

        accuracy = cross_val_score(model, X, y, cv=skf).mean()

        return accuracy
    
    return _objective

study = optuna.create_study(direction='maximize')

objective_func = objective(X, y)  # Closure
study.optimize(objective_func, n_trials=100)

print('Best hyperparameters:')
print('max_depth:', study.best_params['max_depth'])
print('max_features:', study.best_params['max_features'])
print('Highest cross-validation accuracy:', study.best_value)


#b) 
best_params = study.best_params

best_model = DecisionTreeClassifier(**best_params)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print('Test accuracy:', accuracy)


[I 2024-04-01 15:59:43,380] A new study created in memory with name: no-name-cc27a151-22f3-41d7-a1d8-92e314b4aa82
[I 2024-04-01 15:59:43,501] Trial 0 finished with value: 0.7636610701090412 and parameters: {'max_depth': 3, 'max_features': 6}. Best is trial 0 with value: 0.7636610701090412.
[I 2024-04-01 15:59:43,627] Trial 1 finished with value: 0.6981453678423786 and parameters: {'max_depth': 10, 'max_features': 2}. Best is trial 0 with value: 0.7636610701090412.
[I 2024-04-01 15:59:43,907] Trial 2 finished with value: 0.6982111832790396 and parameters: {'max_depth': 7, 'max_features': 13}. Best is trial 0 with value: 0.7636610701090412.
[I 2024-04-01 15:59:44,260] Trial 3 finished with value: 0.656297408222917 and parameters: {'max_depth': 10, 'max_features': 13}. Best is trial 0 with value: 0.7636610701090412.
[I 2024-04-01 15:59:44,555] Trial 4 finished with value: 0.587750909794502 and parameters: {'max_depth': 14, 'max_features': 6}. Best is trial 0 with value: 0.7636610701090412

Best hyperparameters:
max_depth: 3
max_features: 2
Highest cross-validation accuracy: 0.8825285714271037
Test accuracy: 0.8805626824736795


For Q3 and Q4 ,use the following data.

In [48]:
dr=pd.read_csv('https://raw.githubusercontent.com/ogut77/DataScience/main/data/diamond.csv')
dr

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171
...,...,...,...,...,...,...,...,...
5995,1.03,Ideal,D,SI1,EX,EX,GIA,6250
5996,1.00,Very Good,D,SI1,VG,VG,GIA,5328
5997,1.02,Ideal,D,SI1,EX,EX,GIA,6157
5998,1.27,Signature-Ideal,G,VS1,EX,EX,GIA,11206


In [49]:
def Encoder(df):
          from sklearn import preprocessing
          columnsToEncode = list(df.select_dtypes(include=['category','object']))
          le = preprocessing.LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df


In [50]:
dr=Encoder(dr)
dr

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,2,4,2,3,0,1,5169
1,0.83,2,4,3,2,2,0,3470
2,0.85,2,4,2,0,0,1,3183
3,0.91,2,1,2,3,3,1,4370
4,0.83,2,3,2,0,0,1,3171
...,...,...,...,...,...,...,...,...
5995,1.03,2,0,2,0,0,1,6250
5996,1.00,4,0,2,3,3,1,5328
5997,1.02,2,0,2,0,0,1,6157
5998,1.27,3,3,3,0,0,1,11206


In [51]:
y = dr['Price'] #Output
X = dr.drop('Price',axis=1)
X

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report
0,1.10,2,4,2,3,0,1
1,0.83,2,4,3,2,2,0
2,0.85,2,4,2,0,0,1
3,0.91,2,1,2,3,3,1
4,0.83,2,3,2,0,0,1
...,...,...,...,...,...,...,...
5995,1.03,2,0,2,0,0,1
5996,1.00,4,0,2,3,3,1
5997,1.02,2,0,2,0,0,1
5998,1.27,3,3,3,0,0,1


In [55]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)

Q3)Using Linear Regression,Decison Tree Random Forest,XGBoost, Light GBM and Gradient Boosting Classifier with default parameters (no parameter specifications except random_state) calculate R2 statistics on test data. Which method gives the best accuracy on test data

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

r2_scores = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2_scores[name] = r2_score(y_test, y_pred)

best_model = max(r2_scores, key=r2_scores.get)

for name, r2 in r2_scores.items():
    print(f'{name}: R2 = {r2:.4f}')

# Print the best performing model
print(f'\nBest model: {best_model} (R2 = {r2_scores[best_model]:.4f})') #LightGBM (R2 = 0.9813)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 193
[LightGBM] [Info] Number of data points in the train set: 4500, number of used features: 7
[LightGBM] [Info] Start training from score 11827.946667
Linear Regression: R2 = 0.8234
Decision Tree: R2 = 0.9632
Random Forest: R2 = 0.9801
XGBoost: R2 = 0.9809
LightGBM: R2 = 0.9813
Gradient Boosting: R2 = 0.9739

Best model: LightGBM (R2 = 0.9813)


Q4) Using optuna hyperparmeter optimization technique (100 trial)  with Random Forest,XGBoost, Light GBM and Gradient Boosting Regressor

a)find best methods with  parameters  using Cross validation (CV=3) technique for the range of   parameters below. What are the best parameters for the method with highest cross validation R2?

"max_depth": range(2, 7), "max_features": range(2, 7)

 b)Evaluate the performance of the  method with highest cross validation R2 on test data. What is the R2 value?


In [65]:
#a)
import optuna
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

# Define the objective function for optimization
def objective(trial):
    # Define the search space for each hyperparameter
    params = {
        'max_depth': trial.suggest_int('max_depth', 2, 6),
        'max_features': trial.suggest_int('max_features', 2, 6)
    }

    # Select model
    model_name = trial.suggest_categorical('model', ['RandomForestRegressor', 'XGBRegressor', 'LGBMRegressor', 'GradientBoostingRegressor'])
    if model_name == 'RandomForestRegressor':
        model = RandomForestRegressor(**params, random_state=17)
    elif model_name == 'XGBRegressor':
        model = xgb.XGBRegressor(**params, random_state=17)
    elif model_name == 'LGBMRegressor':
        model = LGBMRegressor(**params, random_state=17)
    else:
        model = GradientBoostingRegressor(**params, random_state=17)

    # Perform cross-validation
    cv = KFold(n_splits=3, shuffle=True, random_state=17)
    r2 = cross_val_score(model, X, y, cv=cv, scoring='r2').mean()

    return r2

# Create Optuna study
study = optuna.create_study(direction='maximize')

# Define X and y (your input features and target variable)
X = X
y = y  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=100)

# Get best parameters and model
best_params = study.best_params
best_model_name = best_params['model']

print('Best model:', best_model_name) #XGBRegressor
print('Best hyperparameters:', best_params) #{'max_depth': 4, 'max_features': 5, 'model': 'XGBRegressor'}
print('Highest cross-validation R2:', study.best_value) #0.9857784716376319

#b)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the best model with the best hyperparameters
best_model = None
if best_model_name == 'RandomForestRegressor':
    best_model = RandomForestRegressor(**best_params, random_state=17)
elif best_model_name == 'XGBRegressor':
    best_model = xgb.XGBRegressor(**best_params, random_state=17)
elif best_model_name == 'LGBMRegressor':
    best_model = LGBMRegressor(**best_params, random_state=17)
else:
    best_model = GradientBoostingRegressor(**best_params, random_state=17)

# Train the best model on the entire training dataset
best_model.fit(X_train, y_train)

# Evaluate the best model on the test dataset
y_pred_test = best_model.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)

# Print the R2 value on the test dataset
print(f'R2 value on test data: {r2_test:.4f}') #0.9874


[I 2024-04-01 17:24:39,278] A new study created in memory with name: no-name-bf4fa817-2373-496e-8103-9ad28e74d8df
[I 2024-04-01 17:24:39,624] Trial 0 finished with value: 0.9230614128328277 and parameters: {'max_depth': 2, 'max_features': 2, 'model': 'GradientBoostingRegressor'}. Best is trial 0 with value: 0.9230614128328277.
[I 2024-04-01 17:24:40,316] Trial 1 finished with value: 0.8667450282646745 and parameters: {'max_depth': 3, 'max_features': 5, 'model': 'RandomForestRegressor'}. Best is trial 0 with value: 0.9230614128328277.
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:24:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:24:40] WARNING: C:\buildkite-

[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000100 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 184
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 7
[LightGBM] [Info] Start training from score 11715.579750
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

[I 2024-04-01 17:24:41,774] Trial 6 finished with value: 0.9805282276938386 and parameters: {'max_depth': 4, 'max_features': 2, 'model': 'LGBMRegressor'}. Best is trial 4 with value: 0.9853290610088309.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:24:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:24:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:24:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2

...

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2

[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 184
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 7
[LightGBM] [Info] Start training from score 11715.579750
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2

Best model: XGBRegressor
Best hyperparameters: {'max_depth': 4, 'max_features': 5, 'model': 'XGBRegressor'}
Highest cross-validation R2: 0.9857784716376319
R2 value on test data: 0.9874


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [17:25:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_features", "model" } are not used.

  warnings.warn(smsg, UserWarning)
